# InitialSetup

In [1]:
import os
import shutil

if not os.path.exists('gen_data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if os.path.exists('OUTPUT/'):
    shutil.rmtree('OUTPUT/')
    
os.makedirs('OUTPUT/')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, AveragePooling2D, MaxPooling2D, Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

2023-08-30 11:10:33.826504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading dataset

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)


test_gen = ImageDataGenerator(rescale=1./255)

training_data = train_gen.flow_from_directory('gen_data_set/train', target_size=(256,256), 
                                                 batch_size=32, class_mode='binary')

test_data = test_gen.flow_from_directory('gen_data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

Found 1890 images belonging to 2 classes.
Found 1573 images belonging to 2 classes.


# Training

In [4]:
validation_steps = len(test_data)

# Training model from scratch

In [5]:
classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [6]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [7]:
steps_per_epoch = len(training_data)

classifier.fit(training_data, epochs=90, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=test_data, validation_steps=validation_steps)

Epoch 1/90


2023-08-30 11:10:37.395122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - ETA: 0s - loss: 0.6039 - accuracy: 0.6656

2023-08-30 11:11:27.934118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - 60s 987ms/step - loss: 0.6039 - accuracy: 0.6656 - val_loss: 0.1704 - val_accuracy: 0.9345
Epoch 2/90
60/60 [==============================] - 52s 859ms/step - loss: 0.3320 - accuracy: 0.8661 - val_loss: 0.2925 - val_accuracy: 0.9371
Epoch 3/90
60/60 [==============================] - 50s 837ms/step - loss: 0.2621 - accuracy: 0.8979 - val_loss: 0.2672 - val_accuracy: 0.9421
Epoch 4/90
60/60 [==============================] - 50s 830ms/step - loss: 0.2111 - accuracy: 0.9180 - val_loss: 0.1943 - val_accuracy: 0.9555
Epoch 5/90
60/60 [==============================] - 50s 835ms/step - loss: 0.1912 - accuracy: 0.9265 - val_loss: 0.2420 - val_accuracy: 0.9523
Epoch 6/90
60/60 [==============================] - 50s 830ms/step - loss: 0.1661 - accuracy: 0.9328 - val_loss: 0.1642 - val_accuracy: 0.9625
Epoch 7/90
60/60 [==============================] - 50s 836ms/step - loss: 0.1754 - accuracy: 0.9407 - val_loss: 0.2050 - val_accuracy: 0.9549
Epoch 8/90

60/60 [==============================] - 55s 921ms/step - loss: 0.0573 - accuracy: 0.9804 - val_loss: 0.1282 - val_accuracy: 0.9708
Epoch 59/90
60/60 [==============================] - 56s 931ms/step - loss: 0.0553 - accuracy: 0.9810 - val_loss: 0.0862 - val_accuracy: 0.9765
Epoch 60/90
60/60 [==============================] - 54s 892ms/step - loss: 0.0676 - accuracy: 0.9762 - val_loss: 0.1159 - val_accuracy: 0.9688
Epoch 61/90
60/60 [==============================] - 50s 836ms/step - loss: 0.0739 - accuracy: 0.9720 - val_loss: 0.0828 - val_accuracy: 0.9733
Epoch 62/90
60/60 [==============================] - 50s 839ms/step - loss: 0.0622 - accuracy: 0.9772 - val_loss: 0.0742 - val_accuracy: 0.9758
Epoch 63/90
60/60 [==============================] - 50s 839ms/step - loss: 0.0658 - accuracy: 0.9794 - val_loss: 0.0899 - val_accuracy: 0.9758
Epoch 64/90
60/60 [==============================] - 50s 837ms/step - loss: 0.0537 - accuracy: 0.9810 - val_loss: 0.0854 - val_accuracy: 0.9746
Epoc

In [8]:
classifier.save("OUTPUT/ScratchModel.h5")

In [9]:
_, accuracy = classifier.evaluate(test_data, steps = validation_steps)

 1/50 [..............................] - ETA: 8s - loss: 9.8795e-04 - accuracy: 1.0000

2023-08-30 12:28:29.222365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 8s 167ms/step - loss: 0.1117 - accuracy: 0.9733


In [10]:
print('ScratchModel accuracy :',accuracy)

ScratchModel accuracy : 0.9732994437217712


# Using pre-trained Models

In [11]:
# class Wrapper(tf.keras.Model):
#     def __init__(self, base_model):
#         super(Wrapper, self).__init__()
        
#         self.base_model = base_model
#         self.average_pooling_layer = AveragePooling2D(name="polling")
#         self.flatten = Flatten(name="flatten")
#         self.dense = Dense(64, activation="relu")
#         self.dropout = Dropout(0.5)
#         self.output_layer = Dense(2, activation="softmax")
        
#     def call(self, inputs):
#         x = self.base_model(inputs)
#         x = self.average_pooling_layer(x)
#         x = self.flatten(x)
#         x = self.dense(x)
#         x = self.dropout(x)
#         output = self.output_layer(x)
#         return output

In [12]:
# base_learning_rate = 0.0001
# EPOCHS = 10

## ResNet

In [13]:
# from tensorflow.keras.applications import ResNet152V2

In [14]:
# res_net = ResNet152V2(include_top=False, weights='imagenet', input_shape=(256,256,3))

In [15]:
# res_net.trainable = False
# res_net_classifier = Wrapper(res_net)
# res_net_classifier.compile(optimizer=Adam(learning_rate=base_learning_rate),
#               loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# res_net_classifier.fit(training_data, epochs=EPOCHS, validation_data=test_data, validation_steps=validation_steps)